## Text Classification with ClassifierDL

**Relevant blogpost:** https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32

In [1]:
#### INSTALLATION OF LIBRARIES FOR SPARK
!pip install findspark
!sudo apt update
!sudo apt install openjdk-17-jdk -y
!curl -JLO 'https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz'
!tar xvf spark-3.3.1-bin-hadoop3.tgz
!mv spark-3.3.1-bin-hadoop3 /opt/spark
!pip install pyspark

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]            
Get:3 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [966 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2313 kB]
Get:9 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [774 B]   
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1712 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:13 http://archive.ub

In [2]:
!pip install sparknlp
!pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.6/648.6 kB 56.2 MB/s eta 0:00:00


In [3]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
import os

spark = sparknlp.start(gpu = True)# for GPU training >> sparknlp.start(gpu = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2efde278-0898-4d48-8980-53329a102edb;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.

22/11/21 19:26:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.3
Apache Spark version: 3.3.1


### Load Dataset

In [4]:
!pip install gdown
!mkdir data
!cd data
!gdown 1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
!mv 'Dataset Yelp.zip' data.zip
!unzip data.zip
!mv  -v /notebooks/data/Dataset\ Yelp/* /notebooks/data/
!rm -rf /notebooks/data/Dataset\ Yelp

mkdir: cannot create directory ‘data’: File exists
Downloading...
From: https://drive.google.com/uc?id=1QhtF1UAbVMJyAcqVJDucYwmib0hHCP1A
To: /notebooks/Dataset Yelp.zip
100%|██████████████████████████████████████| 4.23G/4.23G [00:59<00:00, 71.2MB/s]
Archive:  data.zip
   creating: Dataset Yelp/
  inflating: Dataset Yelp/business.json  
  inflating: Dataset Yelp/checkin.json  
  inflating: Dataset Yelp/review.json  
  inflating: Dataset Yelp/tip.json   
  inflating: Dataset Yelp/user.json  
mv: cannot stat '/notebooks/data/Dataset Yelp/*': No such file or directory


In [5]:
#### SPARK
import findspark
findspark.init()
from pyspark.sql import SparkSession
#import pyspark.pandas as ps

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.3")\
    .getOrCreate()

spark.sparkContext.setLogLevel("OFF")

22/11/21 19:33:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)
a = tf.zeros(1)

In [20]:
AllData = spark.read \
      .option("header", True) \
      .json("./data/review.json")

AllData = AllData.select("text")
AllData.show(truncate=15)

+---------------+
|           text|
+---------------+
|If you decid...|
|I've taken a...|
|Family diner...|
|Wow!  Yummy,...|
|Cute interio...|
|I am a long ...|
|Loved this t...|
|Amazingly am...|
|This easter ...|
|Had a party ...|
|My experienc...|
|Locals recom...|
|Love going h...|
|Good food--l...|
|The bun make...|
|Great place ...|
|Tremendous s...|
|The hubby an...|
|I go to blow...|
|My absolute ...|
+---------------+
only showing top 20 rows



In [9]:
from happytransformer import HappyTextClassification

In [14]:
happy_tc = HappyTextClassification(model_type="ROBERTA", model_name="VictorSanh/roberta-base-finetuned-yelp-polarity", num_labels=2)

Some weights of the model checkpoint at VictorSanh/roberta-base-finetuned-yelp-polarity were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
result = happy_tc.classify_text("I hate AI")
print(result.label)

LABEL_0


In [25]:
import pyspark.pandas as ps

In [26]:
reviews = ps.read_json("./data/review.json", lines=True)

/usr/local/lib/python3.9/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [35]:
reviews_tiny = ps.DataFrame(reviews.loc[0:7000, "text"])

In [ ]:
reviews_tiny['score'] = reviews_tiny.apply(lambda x: happy_tc.classify_text(x).label)

In [31]:
def get_label(text):
    result = happy_tc.classify_text(text)
    return result.label

In [32]:
reviews_tiny['label'] = reviews_tiny.apply(get_label)

KeyError: 'text'

In [21]:
(SubSet, OldData) = AllData.randomSplit([0.001,0.999], seed = 100)

In [22]:
SubSet.count()

6932

In [24]:
SubSet.apply(get_label)

AttributeError: 'DataFrame' object has no attribute 'apply'

In [127]:
(trainingData, testData) = SubSet.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 4831
Test Dataset Count: 2101


## ClassifierDL with Universal Sentence Embeddings

In [128]:
# actual content is inside description column
document = DocumentAssembler()\
                .setInputCol("text")\
                .setOutputCol("document")

# we can also use sentece detector here if we want to train on and get predictions for each sentence
use = UniversalSentenceEncoder.pretrained("tfhub_use_lg", "en") \
                .setInputCols("document") \
                .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("stars")\
                .setMaxEpochs(3)\
                .setLr(0.01)\
                .setBatchSize(8)\
                .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(stages = [document,
                                    use,
                                    classsifierdl])

tfhub_use_lg download started this may take some time.
Approximate size to download 753.3 MB
[OK!]


In [129]:
# remove the existing logs
! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory


In [130]:
%%time 
use_pipelineModel = use_clf_pipeline.fit(trainingData)

2022-11-21 03:44:59.839198: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/f63c9d3ee71e_classifier_dl4229645102276336813
2022-11-21 03:44:59.925132: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-11-21 03:44:59.925239: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/f63c9d3ee71e_classifier_dl4229645102276336813
2022-11-21 03:44:59.925502: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03:44:59.927081: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03

Training started - epochs: 3 - learning_rate: 0.01 - batch_size: 8 - training_examples: 4831 - classes: 5
Epoch 1/3 - 1.38s - loss: 884.22864 - acc: 0.4609986 - batches: 604
Epoch 2/3 - 1.09s - loss: 883.5244 - acc: 0.46224236 - batches: 604
Epoch 3/3 - 1.04s - loss: 883.5244 - acc: 0.46224236 - batches: 604
CPU times: user 165 ms, sys: 62 ms, total: 227 ms
Wall time: 1min 41s


In [131]:
log_files = os.listdir("/root/annotator_logs")
log_files

['ClassifierDLApproach_53d031fb416b.log']

In [132]:
with open("/root/annotator_logs/"+log_files[0], "r") as log_file :
    print(log_file.read())

Training started - epochs: 3 - learning_rate: 0.01 - batch_size: 8 - training_examples: 4831 - classes: 5
Epoch 0/3 - 1.38s - loss: 884.22864 - acc: 0.4609986 - batches: 604
Epoch 1/3 - 1.09s - loss: 883.5244 - acc: 0.46224236 - batches: 604
Epoch 2/3 - 1.04s - loss: 883.5244 - acc: 0.46224236 - batches: 604



In [133]:
from sklearn.metrics import classification_report

preds = use_pipelineModel.transform(testData)

preds_df = preds.select('stars','text',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

2022-11-21 03:45:10.650754: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03:45:10.653240: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03:45:10.655734: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03:45:10.656934: I external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 03:45:10.6580

In [134]:
print (classification_report(preds_df['stars'], preds_df['result'].astype(float)))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       333
         2.0       0.00      0.00      0.00       163
         3.0       0.00      0.00      0.00       205
         4.0       0.00      0.00      0.00       467
         5.0       0.44      1.00      0.62       933

    accuracy                           0.44      2101
   macro avg       0.09      0.20      0.12      2101
weighted avg       0.20      0.44      0.27      2101



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Getting prediction from Trained model

In [135]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(use_pipelineModel)

In [137]:
testData.select('text').take(2)

[Row(text='A country town feel inside of New Jersey? Go figure! A gift shop full of knick knacs that your paddywhack cousins adorn their walls with at home. A solid breakfast menu that is perfect for a quick stop or a take out on the go. Enjoyed the service and the escape.'),
 Row(text='Avoid.  The owners are laughing at you.  Didn\'t want to wait 45 min for The Boardwalk Grill,  the Hut was playing loud, poorly covered music, so we dropped into Scully\'s - despite the foul water smell outside.  Service was fine, but all aspects of the food were bad.  Just bad...no heart in this menu at all.   I could get better, fresher seafood in Des Moines for the same price.  "Battered fish" in the F & C we\'re frozen, not fresh & not good.  Fries were cold, though 75% of the menu is fried.  Like Long John Silver\'s, but without the flavor.')]

In [140]:
text='''
Disgusting and ugly place. Won't come back ever. 
'''
result = light_model.annotate(text)

result['class']

['5.0']

In [141]:
light_model.annotate('the soccer games will be postponed.')

{'document': ['the soccer games will be postponed.'],
 'sentence_embeddings': ['the soccer games will be postponed.'],
 'class': ['5.0']}

## Saving & loading back the trained model

In [150]:
use_pipelineModel.stages

ClassifierDLModel_2b9a9179f5ce

In [153]:
# Save a Spark NLP pipeline
use_pipelineModel.save('MyModel1')

# cd into saved dir and zip
! cd ./MyModel1 ; zip -r MyModel1.zip *

  adding: metadata/ (stored 0%)
  adding: metadata/part-00000 (deflated 21%)
  adding: metadata/.part-00000.crc (stored 0%)
  adding: metadata/_SUCCESS (stored 0%)
  adding: metadata/._SUCCESS.crc (stored 0%)
  adding: stages/ (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/ (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/metadata/ (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/metadata/part-00000 (deflated 37%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/metadata/.part-00000.crc (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/metadata/_SUCCESS (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/metadata/._SUCCESS.crc (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/fields/ (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/fields/datasetParams/ (stored 0%)
  adding: stages/2_ClassifierDLModel_2b9a9179f5ce/fields/datasetParams/part-00005 (deflated 26%)
  adding: stages/2_ClassifierDLModel_2

In [ ]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs